## Observations and Insights 

In [55]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

# Study data files
mouse_metadata_path = "Resources/Mouse_metadata.csv"
study_results_path = "Resources/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single dataset
combinedData = pd.merge(mouse_metadata, study_results, on="Mouse ID", how = "outer")
allColumns = combinedData.columns
# Display the data table for preview
combinedData.head()

In [60]:
# Checking the number of mice.
origMouseCount = len(combinedData["Mouse ID"].unique())
print(f"There are {origMouseCount} mice in this study before checking the data")

There are 249 mice in this study before checking the data


In [43]:
#combinedData["Timepoint"].unique()
combinedData["Unique ID"] = combinedData["Mouse ID"].astype(str) + "_" + combinedData["Timepoint"].astype(str)
combinedData.head()

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites,Unique ID
0,k403,Ramicane,Male,21,16,0,45.000000,0,k403_0
1,k403,Ramicane,Male,21,16,5,38.825898,0,k403_5
2,k403,Ramicane,Male,21,16,10,35.014271,1,k403_10
3,k403,Ramicane,Male,21,16,15,34.223992,1,k403_15
4,k403,Ramicane,Male,21,16,20,32.997729,1,k403_20


In [ ]:
dupFinder = pd.DataFrame(combinedData.duplicated(subset=['Mouse ID', 'Timepoint']))
dupFinder.loc[dupFinder[0] == True]

In [45]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
UIDCount = combinedData["Unique ID"].value_counts().astype(str)
UIDCount.head(10)

g989_15    2
g989_0     2
g989_20    2
g989_10    2
g989_5     2
q511_10    1
l490_30    1
x613_15    1
x402_15    1
o331_20    1
Name: Unique ID, dtype: object

In [47]:
# Optional: Get all the data for the duplicate mouse ID. 
dupMouse = combinedData.loc[combinedData["Mouse ID"] == "g989", :]
dupMouse

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites,Unique ID
908,g989,Propriva,Female,21,26,0,45.000000,0,g989_0
909,g989,Propriva,Female,21,26,0,45.000000,0,g989_0
910,g989,Propriva,Female,21,26,5,48.786801,0,g989_5
911,g989,Propriva,Female,21,26,5,47.570392,0,g989_5
912,g989,Propriva,Female,21,26,10,51.745156,0,g989_10
913,g989,Propriva,Female,21,26,10,49.880528,0,g989_10
914,g989,Propriva,Female,21,26,15,51.325852,1,g989_15
915,g989,Propriva,Female,21,26,15,53.442020,0,g989_15
916,g989,Propriva,Female,21,26,20,55.326122,1,g989_20
917,g989,Propriva,Female,21,26,20,54.657650,1,g989_20


In [57]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
cleaned_df = combinedData.loc[combinedData["Mouse ID"] != "g989", allColumns]
cleaned_df.head()

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1


In [59]:
# Checking the number of mice in the clean DataFrame.
cleanedMouseCount = len(cleaned_df["Mouse ID"].unique())
print(f"There are {cleanedMouseCount} mice in this study")

There are 248 mice in this study


## Summary Statistics

In [61]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Use groupby and summary statistical methods to calculate the following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume. 
# Assemble the resulting series into a single summary dataframe.



,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
count,1880.000000,1880.000000,1880.000000,1880.000000,1880.000000
mean,12.757979,25.660106,19.606383,50.435293,1.025000
std,7.181998,3.935064,14.094238,8.914197,1.139971
min,1.000000,15.000000,0.000000,22.050126,0.000000
25%,7.000000,25.000000,5.000000,45.000000,0.000000
50%,13.000000,27.000000,20.000000,48.933454,1.000000
75%,19.250000,29.000000,30.000000,56.324075,2.000000
max,24.000000,30.000000,45.000000,78.567014,4.000000


In [8]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Using the aggregation method, produce the same summary statistics in a single line


## Bar and Pie Charts

In [9]:
# Generate a bar plot showing the total number of unique mice tested on each drug regimen using pandas.



In [10]:
# Generate a bar plot showing the total number of unique mice tested on each drug regimen using pyplot.



In [11]:
# Generate a pie plot showing the distribution of female versus male mice using pandas



In [12]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot



## Quartiles, Outliers and Boxplots

In [13]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse


# Merge this group df with the original dataframe to get the tumor volume at the last timepoint


In [14]:
# Put treatments into a list for for loop (and later for plot labels)


# Create empty list to fill with tumor vol data (for plotting)


# Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
    # Locate the rows which contain mice on each drug and get the tumor volumes
    
    
    # add subset 
    
    
    # Determine outliers using upper and lower bounds
    

In [15]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest


## Line and Scatter Plots

In [16]:
# Generate a line plot of tumor volume vs. time point for a mouse treated with Capomulin


In [17]:
# Generate a scatter plot of average tumor volume vs. mouse weight for the Capomulin regimen


## Correlation and Regression

In [18]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
